In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
!pip install -U accelerate bitsandbytes peft datasets evaluate unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0

In [3]:
##Importing Necessary Libraries
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM
from transformers import DataCollatorWithPadding
from transformers import LlamaConfig,LlamaForSequenceClassification
from transformers import TrainingArguments, Trainer
import peft, accelerate
from transformers import BitsAndBytesConfig
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Force single GPU
# Add at the top of your notebook

import warnings
from pprint import pprint
import math

In [4]:
#Loading Local Datasets - as DatasetDict/Dataset objects.
from datasets import load_dataset
train = {"train":"/kaggle/input/multi-lingual-sentiment-analysis/train.csv"}
test = {"test":"/kaggle/input/multi-lingual-sentiment-analysis/test.csv"}
ds = load_dataset("csv",data_files=train,split="train")
ds_test = load_dataset("csv",data_files=test,split="test")
dataset = ds.map(lambda e: {"text": e["sentence"], "label": e["label"]}, remove_columns=["ID","sentence","label"])
dataset_test = ds_test.map(lambda e: {"text": e["sentence"],"label":""}, remove_columns=["ID","sentence"])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [6]:
model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
tokenizer = AutoTokenizer.from_pretrained(model_id,model_max_length=1024)
# set pad token id
tokenizer.pad_token=tokenizer.eos_token

In [7]:
def tokenize(example):
    example = tokenizer(example['text'],padding=False,truncation=True,)
    return example

In [8]:
def label_it(example):
    if example["label"] == "Negative":
        example["label"] = 0
    if example["label"] == "Positive":
        example["label"] = 1
    return example

dataset = dataset.map(label_it)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
tokenized_ds = dataset.map(tokenize,batched=True,num_proc=12, remove_columns=['text','language'])
print(tokenized_ds)

Map (num_proc=12):   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [10]:
ds_split = tokenized_ds.train_test_split(test_size=0.1,seed=42)
print(ds_split)

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})


In [11]:
# Fix: Rename 'label' column to 'labels' before training
ds_split = ds_split.rename_column("label", "labels")

In [12]:
# dataloader
data_collator = DataCollatorWithPadding(tokenizer,padding=True)

# **Fine Tuning**

In [13]:
#lora
from peft import LoraConfig, TaskType, LoraModel
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    lora_alpha=32,
    lora_dropout=0.05
)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=2,
                                                           pad_token_id=tokenizer.eos_token_id,  quantization_config=quant_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/llama-3.1/transformers/8b-instruct/2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model = model.to(device)

In [16]:
from peft import get_peft_model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()
print(lora_model)

trainable params: 3,407,872 || all params: 7,508,340,736 || trainable%: 0.0454
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Pa

In [17]:
sum(p.numel() for p in lora_model.parameters() if p.requires_grad)  # Should be >0


3407872

In [18]:
import numpy as np
import evaluate
metric = evaluate.load("f1")  # Load F1-score metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")  # Use "macro" for multi-class


In [19]:
num_parameters = 0
for param in lora_model.parameters():   
    if param.requires_grad:
        num_parameters += param.numel()
print(f'Number of Parameters:{num_parameters}')

Number of Parameters:3407872


In [20]:
training_args = TrainingArguments( output_dir='llama3_ft_lora',
                                  eval_strategy="steps",
                                  eval_steps=100,
                                  num_train_epochs=1,
                                  max_steps=1000,
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  bf16=False,
                                  fp16=True,
                                  tf32=False,
                                  gradient_accumulation_steps=4,
                                  adam_beta1=0.9,
                                  adam_beta2=0.999,
                                  learning_rate=2e-5,
                                  weight_decay=0.01,
                                  logging_dir='logs',
                                  logging_strategy="steps",
                                  logging_steps = 100,
                                  save_steps=500,
                                  save_total_limit=20,
                                  report_to='none',
                                )

In [21]:
trainer = Trainer(model=lora_model,
                  args = training_args,
                 train_dataset=ds_split["train"],
                 eval_dataset=ds_split["test"],
                 compute_metrics=compute_metrics,
                 data_collator = data_collator)

In [22]:
results = trainer.train()

Step,Training Loss,Validation Loss,F1
100,1.181400,1.079515,0.524243
200,0.911300,0.686554,0.669703
300,0.577000,0.499892,0.786390
400,0.465900,0.377918,0.839422
500,0.297900,0.326098,0.849865
600,0.148800,0.334655,0.859776
700,0.198600,0.395821,0.888653
800,0.116200,0.390063,0.879227
900,0.170300,0.420094,0.889102
1000,0.101100,0.431626,0.899356


In [23]:
from transformers import TextClassificationPipeline
classifier = TextClassificationPipeline(model=model,
                                       tokenizer=tokenizer,
                                       framework='pt',
                                       task="sentiment-analysis",
                                       )
model.config.id2label = {0:"Negative",1:"Positive"}

Device set to use cuda:0


In [24]:
def pred(example):
    text = example["sentence"]
    prediction = classifier(text)
    example["label"] = prediction[0]["label"]
    return example
ds_test = ds_test.map(pred)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [25]:
sub = {"ID":ds_test["ID"],"label":ds_test["label"]}
sub= pd.DataFrame(sub)
sub.head()

,ID,label
0,1,Negative
1,2,Positive
2,3,Negative
3,4,Positive
4,5,Negative


In [26]:
sub.to_csv("submission.csv",index=False)